In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd

# Recommendation Solutions
<!-- requirement: projects/ml-subset -->
<!--exclude from index-->

1. Develop a "by-category" recommendation model - given a category, return the top rated movies from that category. Keep in mind the above discussion about what "top rated" is. This will also help you look into the data set more closely.

First we will load in all the data and use some Bayesian smoothing to deal with low review counts.  Then we will use a `join` to only pick out the elements we want for each category.

In [ ]:
datadir = 'projects/ml-10M100K' if os.path.isdir('projects/ml-10M100K') else 'projects/ml-subset/'

In [ ]:
def parse_movie_line(l):
    id_, title, cats = l.strip().split('::')
    return {'id': int(id_), 'title': title, 'year': int(title.rsplit(' ')[-1][1:-1]), 
            'categories': cats.split('|')}

with open(os.path.join(datadir, 'movies.dat'), 'r', encoding='utf-8') as f:
    df = pd.DataFrame([parse_movie_line(l) for l in f]).set_index('id')

df_ratings = pd.read_csv(os.path.join(datadir, 'ratings.dat'), sep='::',
                        names=['user_id', 'movie_id','rating','time'],
                        engine='python')

def parse_tag_line(l):
    uid, mid, tag, time = l.strip().split('::')
    return {'user_id': int(uid), 
            'movie_id': int(mid), 
            'tag': tag.lower()}

with open(os.path.join(datadir, 'tags.dat'), 'r', encoding='utf-8') as f:
    df_tags = pd.DataFrame([parse_tag_line(l) for l in f])
    
def bayes_sum_func(N, mu):
    def bayes_sum(x):
         return (x.sum() + mu*N) / (x.count() + N)
    return bayes_sum

In [ ]:
smoothed = df_ratings.groupby("movie_id")["rating"].agg([bayes_sum_func(5, 3), "count"])
def rating_by_category(category):
    df_ans = smoothed.join(df[df['categories'].apply(lambda x: category in x)], how='inner')\
              .sort_values("bayes_sum", ascending=False)[["title", "bayes_sum", "count"]]
    df_ans['category'] = category
    return df_ans

Then we can apply this to each category. We will take the top movie from each and put it into a dataframe.

In [ ]:
top_dict = {category: rating_by_category(category).iloc[:1]
  for category in set(df['categories'].sum())}

pd.concat(top_dict.values()).set_index('category')

1. At the beginning of this notebook, we discussed Bayesian smoothing. Use Bayesian smoothing to appropriately sort the cooperative recommendations directly above.

All we need to do is aggregate by the `bayes_sum` function.

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.neighbors import NearestNeighbors

In [ ]:
by_user_ratings = df_ratings.groupby('user_id').apply(
    lambda items: {i[1]: i[2] for i in items.itertuples(index=False)})
features = DictVectorizer().fit_transform(by_user_ratings)

nn = NearestNeighbors(n_neighbors=20, metric='cosine', algorithm='brute').fit(features)

In [ ]:
uid = 9689
df_ratings_title = df_ratings.merge(df[['title']], left_on='movie_id', right_index=True)
user_row = features[by_user_ratings.index.get_loc(uid), :]
dists, indices = nn.kneighbors(user_row)
neighbors = [by_user_ratings.index[i] for i in indices[0]][1:] # value at index 0 is our user themself
ratings_grp = df_ratings_title[df_ratings_title['user_id'].isin(neighbors)] \
    .groupby('title')['rating']
print("{} recommendations".format(len(ratings_grp)))
ratings_grp.agg(['mean', 'count', bayes_sum_func(2, 3)]).sort_values('bayes_sum', ascending=False).head(5)

4. In weighting reviews, we consider not reviewing a movie to be less of a recommendation that scoring it a one.  Shift the rating scale to change this, and see how the affects the resultant recommendations.

The easiest way to shift the reviews is to shift every review by -2.5 such that the reviews not seen are an "average" zero.

In [ ]:
df_ratings['rating_shifted'] = df_ratings['rating'] - 2.5
df_ratings.head()

1. Implement another user-item interaction model.  Here are a few ideas:
   - Measure how similar the user is to fans of the item, and regress against that distance.
   - Measure how similar the item is to items the user likes, and regress against that distance.
   - Use a nearest neighbors approach to report the scores given to an item by users similar to the user in question.
   
2. Blend two (or more) interaction models together.  These can be different algorithms from (1) or several matrix factorization algorithms with different hyperparameters.  Linear regression is a good way to automatically perform a weighted average together.  The `BellKor` team had success with **gradient-boosted trees** as blenders.

3. Add features of the movie or user to the blend.  A nonlinear model, such as **random forest** or gradient-boosted trees, may be able to detect that one of the component models does particularly well for a certain subset of features.

Let's implement another user item interaction model. We will choose number three here.   Let's look for similar users, we will do this by reusing our previous code and the trained nearest neighbors model.  We will train the model to select 50 neighbors.

In [ ]:
nn = NearestNeighbors(n_neighbors=50, metric='cosine', algorithm='brute').fit(features)

In [ ]:
N_USERS = df_ratings['user_id'].unique().shape[0]
def compute_rating(uid, movie_id):
    user_row = features[by_user_ratings.index.get_loc(uid), :]
    dists, indices = nn.kneighbors(user_row)
    neighbors = [by_user_ratings.index[i] for i in indices[0]][1:] # value at index 0 is our user themself
    df = df_ratings.join(pd.DataFrame({
        'dists':1.0/dists[0],
        'ind':indices[0]
    }).set_index('ind'), how='inner')
    df = df[df['movie_id'] == movie_id]
    if df.shape[0] == 0:
        return 2 # handle case where no near neighbors
    return (df['dists'] * df['rating']).sum() / df['dists'].sum()

In [ ]:
compute_rating(uid, 2)

Now we will combine two interaction models together. We will train them here.

In [ ]:
from sklearn import base
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import Ridge
class Dictizer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col):
        self.col = col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.col].apply(lambda x: {x: 1})
    
user_dict = Pipeline([('dict', Dictizer('user_id')),
                      ('vect', DictVectorizer())])
movie_dict = Pipeline([('dict', Dictizer('movie_id')),
                       ('vect', DictVectorizer())])
union = FeatureUnion([('user_dict', user_dict),
                      ('movie_dict', movie_dict)])
lr_pipe = Pipeline([('union', union),
                    ('lr', Ridge(alpha=3.1622))]) # obtained in lecture

# Algorithm adapted from http://bugra.github.io/work/notes/2014-04-19/alternating-least-squares-method-for-collaborative-filtering/
# Also note, there is (slow) movement to integrate something into Scikit-Learn 
# https://github.com/scikit-learn/scikit-learn/issues/6142
class ResidualMatrixFactorization(base.BaseEstimator, base.RegressorMixin):
    
    def __init__(self, base_est, n_factors, n_iters, alpha):
        self.base_est = base_est
        self.n_factors = n_factors
        self.n_iters = n_iters
        self.alpha = alpha
        
        self.user_index = self.rating_index = self.Q = self.W = self.user_f = self.movie_f = None
    
    def fit_step(self):
        if self.user_index is None:
            self.init_fit(X, y)
        
        alphas = self.alpha * np.eye(self.n_factors)
        for u, Wu in enumerate(self.W):
            movie_f = self.movie_f[:,Wu]
            self.user_f[u] = np.linalg.solve(
                np.dot(movie_f, movie_f.T) + alphas,
                np.dot(movie_f, self.Q[u, Wu])).T
        for i, Wi in enumerate(self.W.T):
            user_f = self.user_f[Wi,:]
            self.movie_f[:, i] = np.linalg.solve(
                np.dot(user_f.T, user_f) + alphas,
                np.dot(user_f.T, self.Q[Wi, i]))
        
        return self
    
    def fit(self, X, y):
        self.base_est.fit(X, y)
        residuals = y - self.base_est.predict(X)
        df = pd.DataFrame({'user_id': X['user_id'], 'movie_id': X['movie_id'], 'residuals': residuals})
        
        rating_mat = df.pivot_table('residuals', 'user_id', 'movie_id')
        self.user_index = rating_mat.index
        self.movie_index = rating_mat.columns
        
        self.Q = rating_mat.fillna(0).values
        self.W = (~rating_mat.isnull()).values
        self.user_f = np.random.rand(self.Q.shape[0], self.n_factors)
        self.movie_f = np.random.rand(self.n_factors, self.Q.shape[1])

        for n in range(self.n_iters):
            self.fit_step()
        
        return self
    
    def interaction_prediction(self, row):
        uid = row['user_id']
        mid = row['movie_id']
        if uid in self.user_index and mid in self.movie_index:
            return np.dot(self.user_f[self.user_index.get_loc(uid), :],
                          self.movie_f[:, self.movie_index.get_loc(mid)])
        return 0
        
    def predict(self, X):
        return self.base_est.predict(X) + X.apply(self.interaction_prediction, axis=1).values

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
users = df_ratings['user_id'].unique()
np.random.seed(42)
user_sample = np.random.choice(users, 3000)
df_ratings_sample = df_ratings[df_ratings['user_id'].isin(user_sample)]
X_train, X_test, y_train, y_test = train_test_split(df_ratings_sample, df_ratings_sample['rating'])

In [ ]:
X_train.head()

In [ ]:
int_1 = ResidualMatrixFactorization(lr_pipe, 20, 10, 30)

In [ ]:
int_2 = ResidualMatrixFactorization(lr_pipe, 10, 10, 30)

Now we will need to write a `Transformer` to take the output of the `predict` method of the previous models and use it in a `transform` method so we can use these in a pipeline.

In [ ]:
class TransformModel(base.BaseEstimator, base.TransformerMixin):
    def __init__(self, model):
        self.model = model
        
    def fit(self, X, y):
        self.model.fit(X, y)
        return self
    
    def transform(self, X):
        return [[i] for i in self.model.predict(X)] # needs to be a matrix

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
mod = Pipeline([
    ('union', FeatureUnion([
        ('in 1', TransformModel(int_1)),
        ('in 2', TransformModel(int_2))
    ])),
    ('fit', GradientBoostingRegressor())
])

In [ ]:
mod.fit(X_train, y_train)

Now adding other features is pretty easy, we just add them directly to the `FeatureUnion` in the blend.  We will leave that one to you.

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*